# Nighttime Light

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import rioxarray
import xarray
from pathlib import Path
import os
from tqdm import tqdm
from multiprocessing import Pool
from geocube.api.core import make_geocube
%matplotlib inline
import time
import workers

In [2]:
def lightter(out_grid):
    grouped_elevation = out_grid.drop("spatial_ref").groupby(out_grid.group)
    grid_mean = grouped_elevation.mean().rename({"light": "light_mean"})
    grid_min = grouped_elevation.min().rename({"light": "light_min"})
    grid_max = grouped_elevation.max().rename({"light": "light_max"})
    grid_std = grouped_elevation.std().rename({"light": "light_std"})
    # print(grid_mean, grid_min, grid_max, grid_std)
    zonal_stats = xarray.merge([grid_mean, grid_min, grid_max, grid_std]).to_dataframe()
    # zonal_stats['base']=b
    # zonal_stats['ctrl_dis']=d
    # print(zonal_stats)
    return zonal_stats

In [33]:
def get_light(fpath):
    # fpath = "D:\Projects\CityGovRelocation\0.DataCleaning\1.Input\Light\PANDA_China_2000.tif"
    lightr = rioxarray.open_rasterio(fpath, masked=True)
    light = lightr.rio.clip(comms.geometry.values, comms.crs, from_disk=True).sel(band=1).drop("band")
    light.name = "light"
    
    zonal_stats_l = []
    vector_data_l = []
    for d in [1000,1250, 1500, 1750,2000,2250, 2500,2750,3000]:
        for b in range(1, 13):
            vector_data=comms.loc[ (comms['base']==b) & (comms['ctrl_dis'] ==d) ,:]
            # print(d, b, )
            out_grid = make_geocube(vector_data=vector_data, measurements=["group"], like=light,)
            out_grid["light"] = (light.dims, light.values, light.attrs, light.encoding)
            out_grid['base'] = b
            out_grid['ctrl_dis'] = d
            # print(out_grid['base'].values)
            vector_data_l.append(out_grid)

    if __name__ == '__main__':
        with Pool(os.cpu_count()) as pool:
            for op in pool.map(workers.lightter, vector_data_l):
                zonal_stats_l.append(op)
    #
    df_light = pd.concat(zonal_stats_l)
    df_light["file"] = fpath.stem
    
    # df_light = df_light.merge(comms, on="group")
    print(len(df_light), len(comms))
    return df_light

In [34]:
comms = gpd.read_file("../1.Input/comms1.geojson")
comms['group'] = comms.index.astype('int')
comms['group'] = comms['group'] +1
comms['base'] = np.tile(np.arange(1, 13), len(comms))[:len(comms)]

dpath = "../1.Input/Light"

for (root, dirs, filenames) in os.walk(dpath, topdown=False):
    for _file in filenames:
        print(_file)
        fpath = Path(root).joinpath(_file)
        if fpath.suffix in ['.tif', 'tiff'] and not Path(f"../2.Output/{fpath.stem}_light_1.csv").is_file():
            start = time.time()
            df = get_light(fpath)
            df.to_csv(f"../2.Output/{fpath.stem}_light_1.csv")
            end = time.time()
            print(end - start)

PANDA_China_1984.tif
3456 3456
212.40863132476807
PANDA_China_1985.tif
PANDA_China_1986.tif
PANDA_China_1987.tif
PANDA_China_1988.tif
PANDA_China_1989.tif
PANDA_China_1990.tif
PANDA_China_1991.tif
PANDA_China_1992.tif
PANDA_China_1993.tif
PANDA_China_1994.tif
PANDA_China_1995.tif
PANDA_China_1996.tif
PANDA_China_1997.tif
PANDA_China_1998.tif
PANDA_China_1999.tif
PANDA_China_2000.tif
PANDA_China_2001.tif
PANDA_China_2002.tif
PANDA_China_2003.tif
PANDA_China_2004.tif
PANDA_China_2005.tif
PANDA_China_2006.tif
PANDA_China_2007.tif
PANDA_China_2008.tif
PANDA_China_2009.tif
PANDA_China_2010.tif
PANDA_China_2011.tif
PANDA_China_2012.tif
PANDA_China_2013.tif
PANDA_China_2014.tif
PANDA_China_2015.tif
PANDA_China_2016.tif
PANDA_China_2017.tif
PANDA_China_2018.tif
PANDA_China_2019.tif
PANDA_China_2020.tif


In [ ]:
comms = gpd.read_file("../1.Input/comms2.geojson")
comms['group'] = comms.index.astype('int')
comms['group'] = comms['group'] +1
comms['base'] = np.tile(np.arange(1, 13), len(comms))[:len(comms)]

dpath = "../1.Input/Light"

for (root, dirs, filenames) in os.walk(dpath, topdown=False):
    for _file in filenames:
        print(_file)
        fpath = Path(root).joinpath(_file)
        if fpath.suffix in ['.tif', 'tiff'] and not Path(f"../2.Output/{fpath.stem}_light_2.csv").is_file():
            start = time.time()
            df = get_light(fpath)
            df.to_csv(f"../2.Output/{fpath.stem}_light_2.csv")
            end = time.time()
            print(end - start)

In [ ]:
comms = gpd.read_file("../1.Input/comms3.geojson")
comms['group'] = comms.index.astype('int')
comms['group'] = comms['group'] +1
comms['base'] = np.tile(np.arange(1, 13), len(comms))[:len(comms)]

dpath = "../1.Input/Light"

for (root, dirs, filenames) in os.walk(dpath, topdown=False):
    for _file in filenames:
        print(_file)
        fpath = Path(root).joinpath(_file)
        if fpath.suffix in ['.tif', 'tiff'] and not Path(f"../2.Output/{fpath.stem}_light_3.csv").is_file():
            start = time.time()
            df = get_light(fpath)
            df.to_csv(f"../2.Output/{fpath.stem}_light_3.csv")
            end = time.time()
            print(end - start)